# Implementation
Authors: Anders Logg and Hans Petter Langtangen

Adaptation to dolfin-X: Jørgen S. Dokken

## Test problem
To solve a test problem, we need to choose the right hand side $f$ and the coefficient $q(u)$ and the boundary $u_D$. Previously, we have worked with manufactured solutions that can  be reproduced without approximation errors. This is more difficult in non-linear porblems, and the algebra is more tedious. Howeve, we will utilize UFLs differentiation capabilities to obtain a manufactured solution. 

For this problem, we will choose $q(u) = 1 + u^2$ and define a two dimensional manufactured solution that is linear in $x$ and $y$:

In [1]:
import dolfinx
import ufl
from mpi4py import MPI

def q(u):
    return 1 + u**2

mesh = dolfinx.UnitSquareMesh(MPI.COMM_WORLD, 10, 10)
x = ufl.SpatialCoordinate(mesh)
u_ufl = 1 + x[0] + 2*x[1]
f = - ufl.div(q(u_ufl)*ufl.grad(u_ufl))

Note that since `x` is a 2D vector, the first component (index 0) resemble $x$, while the second component (index 1) resemble $y$. The resulting function `f` can be directly used in variational formulations in dolfin-X.

As we now have defined our source term and exact solution, we can create the appropriate function space and boundary conditions.
Note that as we have already defined the exact solution, we only have to convert it to a python function that can be evaluated in the interpolation function. We do this by employing the Python `eval` and `lambda`-functions. 

In [2]:
from petsc4py import PETSc
import numpy
import dolfinx.fem
V = dolfinx.FunctionSpace(mesh, ("CG", 1))
u_exact = lambda x: eval(str(u_ufl))
u_D = dolfinx.Function(V)
u_D.interpolate(u_exact)
u_D.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
fdim = mesh.topology.dim - 1
boundary_facets = dolfinx.mesh.locate_entities_boundary(
    mesh, fdim, lambda x: numpy.full(x.shape[1], True, dtype=numpy.bool))
bc = dolfinx.DirichletBC(u_D, dolfinx.fem.locate_dofs_topological(V, fdim, boundary_facets))

We are now ready to define the variational formulation. Note that as the problem is non-linear, we have replace the `TrialFunction` with a `Function`.

In [3]:
uh = dolfinx.Function(V)
v = ufl.TestFunction(V)
F = q(uh)*ufl.dot(ufl.grad(uh), ufl.grad(v))*ufl.dx - f*v*ufl.dx

The next step is to solve the non-linear problem. To do so, we use [Newtons method](https://en.wikipedia.org/wiki/Newton%27s_method). We start by creating a class containing the core functions that are required. Note that we keep a cache of the residual vector `_F` and the matrix `_J`. This is to reduce duplication of objects if the solver would be used multiple times, for instance for a time-dependent problem.

In [4]:
import dolfinx.cpp
class NonlinearPDEProblem(dolfinx.cpp.nls.NonlinearProblem):
    """Nonlinear problem class for a PDE problem."""

    def __init__(self, F, u, bcs):
        super().__init__()
        V = u.function_space
        du = ufl.TrialFunction(V)
        self.L = F
        self.a = ufl.derivative(F, u, du)
        self.bcs = bcs
        self._F, self._J = None, None

    def form(self, x):
        x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    def F(self, x):
        """Assemble residual vector."""
        if self._F is None:
            self._F = dolfinx.fem.assemble_vector(self.L)
        else:
            with self._F.localForm() as f_local:
                f_local.set(0.0)
            self._F = dolfinx.fem.assemble_vector(self._F, self.L)
        dolfinx.fem.apply_lifting(self._F, [self.a], [self.bcs], [x], -1.0)
        self._F.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        dolfinx.fem.set_bc(self._F, self.bcs, x, -1.0)

        return self._F

    def J(self, x):
        """Assemble Jacobian matrix."""
        if self._J is None:
            self._J = dolfinx.fem.assemble_matrix(self.a, self.bcs)
        else:
            self._J.zeroEntries()
            self._J = dolfinx.fem.assemble_matrix(self._J, self.a, self.bcs)
        self._J.assemble()
        return self._J


As we have now defined the the solution strategy, we can use the built-in Newton-solver in dolfin-X to solve the variational problem. We can set the convergence criterions for the solver by changing the absolute tolerance (`atol`), relative tolerance (`rtol`) or the convergence criterion (`residual` or `incremental`).

In [5]:
# Create nonlinear problem
problem = NonlinearPDEProblem(F, uh, [bc])

# Create Newton solver and solve
solver = dolfinx.cpp.nls.NewtonSolver(MPI.COMM_WORLD)
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "residual"
dolfinx.log.set_log_level(dolfinx.log.LogLevel.INFO)
n, converged = solver.solve(problem, uh.vector)
assert(converged)
print("Number of interations: {0:d}".format(n))

Number of interations: 8


We observe that the solver converges after $8$ iterations.
If we think of the problem in terms of finite differences on a uniform mesh, $\mathcal{P}_1$ elements mimic standard second-order finite differences, which compute the derivative of a linear or quadratic funtion exactly. Here $\nabla u$ is a constant vector, which is multiplied by $1+u^2$, which is a second order polynomial in $x$ and $y$, which the finite difference operator would compute exactly. We can therefore, even with $\mathcal{P}_1$ elements, expect the manufactured solution to be reproduced by the numerical method. However, if we had chosen a nonlinearity, such as $1+u^4$, this would not be the case, and we would need to verify convergence rates.

In [6]:
# Compute L2 error and error at nodes
V_ex = dolfinx.FunctionSpace(mesh, ("CG", 2))
u_ex = dolfinx.Function(V_ex)
u_ex.interpolate(u_exact)
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
error_L2 = numpy.sqrt(dolfinx.fem.assemble_scalar((uh - u_ex)**2 * ufl.dx))
print("L2-error: {0:.2e}".format(error_L2))

# Compute values at mesh vertices
u_vertex_values = uh.compute_point_values()
u_ex_vertex_values = u_ex.compute_point_values()
error_max = numpy.max(numpy.abs(u_vertex_values - u_ex_vertex_values))
print("Error_max = {0:.2e}".format(error_max))

L2-error: 3.92e-16
Error_max = 8.88e-16
